In [1]:
# Import libraries
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm

In [2]:
# Create hopteaming database.
db = sqlite3.connect('data/hopteaming.sqlite')
cursor = db.cursor()
cursor.executescript("""
DROP TABLE IF EXISTS hopteaming;
DROP TABLE IF EXISTS nppes;
DROP TABLE IF EXISTS affiliations;
DROP TABLE IF EXISTS hospital_referrals;
""")

In [3]:
# Create list of columns to use for nppes dataset.
list_of_nppes_cols = ['NPI',
                      'Entity Type Code',
                      'Provider Organization Name (Legal Business Name)',
                      'Provider Last Name (Legal Name)',
                      'Provider First Name',
                      'Provider Middle Name',
                      'Provider Name Prefix Text',
                      'Provider Name Suffix Text',
                      'Provider Credential Text',
                      'Provider First Line Business Practice Location Address',
                      'Provider Second Line Business Practice Location Address',
                      'Provider Business Practice Location Address City Name',
                      'Provider Business Practice Location Address State Name',
                      'Provider Business Practice Location Address Postal Code',
                      'Healthcare Provider Taxonomy Code_1',
                      'Healthcare Provider Primary Taxonomy Switch_1',
                      'Healthcare Provider Taxonomy Code_2',
                      'Healthcare Provider Primary Taxonomy Switch_2',
                      'Healthcare Provider Taxonomy Code_3',
                      'Healthcare Provider Primary Taxonomy Switch_3',
                      'Healthcare Provider Taxonomy Code_4',
                      'Healthcare Provider Primary Taxonomy Switch_4',
                      'Healthcare Provider Taxonomy Code_5',
                      'Healthcare Provider Primary Taxonomy Switch_5',
                      'Healthcare Provider Taxonomy Code_6',
                      'Healthcare Provider Primary Taxonomy Switch_6',
                      'Healthcare Provider Taxonomy Code_7',
                      'Healthcare Provider Primary Taxonomy Switch_7',
                      'Healthcare Provider Taxonomy Code_8',
                      'Healthcare Provider Primary Taxonomy Switch_8',
                      'Healthcare Provider Taxonomy Code_9',
                      'Healthcare Provider Primary Taxonomy Switch_9',
                      'Healthcare Provider Taxonomy Code_10',
                      'Healthcare Provider Primary Taxonomy Switch_10',
                      'Healthcare Provider Taxonomy Code_11',
                      'Healthcare Provider Primary Taxonomy Switch_11',
                      'Healthcare Provider Taxonomy Code_12',
                      'Healthcare Provider Primary Taxonomy Switch_12',
                      'Healthcare Provider Taxonomy Code_13',
                      'Healthcare Provider Primary Taxonomy Switch_13',
                      'Healthcare Provider Taxonomy Code_14',
                      'Healthcare Provider Primary Taxonomy Switch_14',
                      'Healthcare Provider Taxonomy Code_15',
                      'Healthcare Provider Primary Taxonomy Switch_15']

# List of columns to pivot NPPES dataset on.
pivot_cols = ['Healthcare Provider Taxonomy Code_1',
            'Healthcare Provider Primary Taxonomy Switch_1',
            'Healthcare Provider Taxonomy Code_2',
            'Healthcare Provider Primary Taxonomy Switch_2',
            'Healthcare Provider Taxonomy Code_3',
            'Healthcare Provider Primary Taxonomy Switch_3',
            'Healthcare Provider Taxonomy Code_4',
            'Healthcare Provider Primary Taxonomy Switch_4',
            'Healthcare Provider Taxonomy Code_5',
            'Healthcare Provider Primary Taxonomy Switch_5',
            'Healthcare Provider Taxonomy Code_6',
            'Healthcare Provider Primary Taxonomy Switch_6',
            'Healthcare Provider Taxonomy Code_7',
            'Healthcare Provider Primary Taxonomy Switch_7',
            'Healthcare Provider Taxonomy Code_8',
            'Healthcare Provider Primary Taxonomy Switch_8',
            'Healthcare Provider Taxonomy Code_9',
            'Healthcare Provider Primary Taxonomy Switch_9',
            'Healthcare Provider Taxonomy Code_10',
            'Healthcare Provider Primary Taxonomy Switch_10',
            'Healthcare Provider Taxonomy Code_11',
            'Healthcare Provider Primary Taxonomy Switch_11',
            'Healthcare Provider Taxonomy Code_12',
            'Healthcare Provider Primary Taxonomy Switch_12',
            'Healthcare Provider Taxonomy Code_13',
            'Healthcare Provider Primary Taxonomy Switch_13',
            'Healthcare Provider Taxonomy Code_14',
            'Healthcare Provider Primary Taxonomy Switch_14',
            'Healthcare Provider Taxonomy Code_15',
            'Healthcare Provider Primary Taxonomy Switch_15']

# Dictionary for renaming columns in merged dataset.
rename_cols_dict = {'provider_organization_name_(legal_business_name)':'organization_name',
                    'provider_name_prefix_text':'name_prefix',
                    'provider_first_name':'first_name',
                    'provider_middle_name':'middle_name',
                    'provider_last_name_(legal_name)':'last_name',
                    'provider_name_suffix_text':'name_suffix',
                    'provider_credential_text':'credentials',
                    'provider_first_line_business_practice_location_address':'address_first_line',
                    'provider_second_line_business_practice_location_address':'address_second_line',
                    'provider_business_practice_location_address_city_name':'city',
                    'provider_business_practice_location_address_state_name':'state',
                    'provider_business_practice_location_address_postal_code':'zipcode',
                    'code':'taxonomy_code',
                    'grouping':'taxonomy_code_grouping',
                    'classification':'taxonomy_code_classification',
                    'specialization':'taxonomy_code_specialization',
                    'definition':'taxonomy_code_definition',
                    'display_name':'taxonomy_code_display_name',
                    'section':'taxonomy_code_section'}


# (preliminary) Final column ordering.
final_column_ordering = ['npi', 
                         'entity_type_code', 
                         'organization_name',
                         'name_prefix', 
                         'first_name', 
                         'middle_name',
                         'last_name',
                         'name_suffix',
                         'credentials', 
                         'address_first_line', 
                         'address_second_line', 
                         'city',
                         'state', 
                         'zipcode', 
                         'cbsa',
                         'taxonomy_code',
                         'taxonomy_code_grouping',
                         'taxonomy_code_classification',
                         'taxonomy_code_specialization',
                         'taxonomy_code_display_name',
                         'taxonomy_code_section']

# Read in taxonomy code classification dataset.
classifications = pd.read_csv('data/nucc_taxonomy_220.csv')

# Read in zip to cbsa dataset. Change zipcode column to string for ease of merge later.
cbsa = pd.read_csv('data/ZIP_CBSA_122021.csv', usecols = ['zip','cbsa'])
cbsa['zip'] = cbsa['zip'].astype(str)
cbsa = cbsa.loc[cbsa['cbsa'] == 34980]

In [4]:
# Add in affiliations
list_of_dfs = []

for chunk in tqdm(pd.read_csv('data/DAC_NationalDownloadableFile.csv', 
                              chunksize = 10000, 
                              low_memory = False,
                              encoding ='ansi')):
    
    chunk = chunk.loc[(chunk[' st'] == 'TN') & (chunk['NPI'].notnull())]
    
    list_of_dfs.append(chunk)
    
affiliations = pd.concat(list_of_dfs)

# Upload to database.
affiliations.to_sql('affiliations', 
                    db, 
                    if_exists = 'append', 
                    index = False)

# Write dataframe to csv.
affiliations.to_csv('data/affiliations.csv', index = False)

0it [00:00, ?it/s]

In [5]:
# Read in chunks from nppes csv file as chunks.
header = True
for chunk in tqdm(pd.read_csv('data/npidata_pfile_20050523-20220213.csv', 
                              usecols = list_of_nppes_cols, 
                              chunksize = 100000,
                              low_memory = False)):
    
    # Filter chunk for practice locations in TN only.
    chunk = chunk[chunk['Provider Business Practice Location Address State Name'] == 'TN']
    

    # Create two subsets of nppes dataset pivoted by NPI with taxonomy switches and codes and then concatenate together into
    # larger subset.
    chunk_taxonomy_code_subset = (
        pd.concat([chunk.melt(id_vars = 'NPI',
                              value_vars = pivot_cols[0:29:2],
                              var_name = 'code_number',
                              value_name = 'code')\
                   .sort_values(by = 'NPI'),
                   chunk.melt(id_vars = 'NPI',
                              value_vars = pivot_cols[1:29:2],
                              var_name = 'switch_number',
                              value_name = 'switch')\
                   .sort_values(by = 'NPI')],
                  axis = 1)
              )

    # Merge the subset dataset back into the larger one.
    chunk = (
        chunk\
        .drop(pivot_cols, axis = 1)\
        .merge(chunk_taxonomy_code_subset[['NPI','code']]\
        .loc[chunk_taxonomy_code_subset['switch'] == 'Y']\
        .iloc[:,[0,2]]\
        .sort_values('NPI')\
        .set_index('NPI')\
        .reset_index(), 
               how = 'inner', 
               on = 'NPI')
             )
    
    # Change zipcode to string to trim.
    chunk['Provider Business Practice Location Address Postal Code'] = (
        chunk['Provider Business Practice Location Address Postal Code']\
        .astype(str)\
        .apply(lambda x: x[0:5])
    )
    
    # Merge cbsa and classifications dataset to the chunk.
    chunk = (
        chunk.merge(classifications,
                    left_on = 'code',
                    right_on = 'Code',
                    how = 'inner')\
        .merge(cbsa,
               left_on = 'Provider Business Practice Location Address Postal Code',
               right_on = 'zip',
               how = 'inner')\
        .drop(columns = ['Code',
                         'Notes',
                         'zip'])
    )
    
    # Rename and reorder all columns.
    chunk.columns = [column.lower().replace(' ', '_') for column in chunk.columns]
    
    chunk = (
        chunk\
        .rename(columns = rename_cols_dict)
    )
    
    chunk = chunk[final_column_ordering]
    
    # Read chunk into sqlite database.
    chunk.to_sql('nppes', 
                 db, 
                 if_exists = 'append', 
                 index = False)
    
    # Write chunk to csv.
    chunk.to_csv('data/nppes.csv', 
                 header = header, 
                 mode = 'a', 
                 index = False)
    header = False

0it [00:00, ?it/s]

In [6]:
# Read back in full nppes dataset but only two rows--npi and entity type code.
only_npi_and_et = pd.read_csv('data/nppes.csv', 
                              usecols = ['npi','entity_type_code'], 
                              dtype = ('int','int'))

# Split these into two datasets for each npi column in hopteaming.
from_npi_et = (
    only_npi_and_et[['npi','entity_type_code']]\
    .rename(columns = {'entity_type_code':'from_npi_et'})\
    .set_index('npi')
)
to_npi_et = (
    only_npi_and_et[['npi','entity_type_code']]\
    .rename(columns = {'entity_type_code':'to_npi_et'})\
    .set_index('npi')
)

# Create dictionary to map values for city filter.
cbsa_dict_filter = dict(pd.read_csv('data/nppes.csv', 
                                    usecols = ['npi','cbsa'],
                                    dtype = {'npi':'int'}).values)

In [7]:
# Read in chunks from hop_teaming csv file. 
header = True
for chunk in tqdm(pd.read_csv('data/DocGraph_Hop_Teaming_2018.csv', chunksize = 100000)):
    
    # Merge chunk with npi entity codes.
    chunk = (
        chunk\
        .set_index('from_npi')\
        .join(from_npi_et,
              how = 'inner')\
        .reset_index()\
        .rename(columns = {'index':'from_npi'})\
        .set_index('to_npi')\
        .join(to_npi_et,
              how = 'inner')\
        .reset_index()\
        .rename(columns = {'index':'to_npi'})
       )
    
    # Filter based on entity codes.
    chunk = (
        chunk[(chunk['to_npi_et'] == 2) & 
              (chunk['from_npi_et'] == 1)]\
        .drop(columns = ['to_npi_et','from_npi_et'])
    )
    
    # Filter for transaction count and average day wait and read to sqlite.
    chunk = chunk[(chunk['transaction_count'] >= 50) & 
                  (chunk['average_day_wait'] < 50) & 
                 (chunk['to_npi'].map(cbsa_dict_filter) == 34980) &
                 (chunk['from_npi'].map(cbsa_dict_filter) == 34980)]
    
    # Write chunk to database.
    chunk.to_sql('hopteaming', 
                 db, 
                 if_exists = 'append', 
                 index = False)  
    
    # Write chunk to csv.
    chunk.to_csv('data/hopteaming.csv', 
                 header = header, 
                 mode = 'a', 
                 index = False)
    header = False

0it [00:00, ?it/s]

In [8]:
# Read back in hopteaming and npi datasets.
hopteaming = pd.read_csv('data/hopteaming.csv')
nppes = pd.read_csv('data/nppes.csv')

In [9]:
# Merge them together with information for each referral party.
hospital_referrals = (
    hopteaming\
    .merge(nppes,
           left_on = 'from_npi',
           right_on = 'npi',
           how = 'inner')\
    .merge(nppes,
           left_on = 'to_npi',
           right_on = 'npi',
           how = 'inner',
           suffixes = ('_nppes_referral_from', 
                       '_nppes_referral_to'))
)

In [10]:
# Exclude certain specialties.
exclude_these = ['Diagnostic Radiology Physician',
                 'Therapeutic Radiology Physician',
                 'Neuroradiology Physician',
                 'Diagnostic Ultrasound Physician',
                 'Registered Dietitian',
                 'Nuclear Radiology Physician',
                 'Pediatric Radiology Physician',
                 'Internist Chiropractor',
                 'Independent Medical Examiner Physician',
                 'Oral and Maxillofacial Surgery (Dentist)',
                 'Clinical Medical Laboratory',
                 'Land Ambulance',
                 'Oxygen Equipment & Supplies (DME)',
                 'Ambulance',
                 'Community/Retail Pharmacy',
                 'Radiology Clinic/Center',
                 'Durable Medical Equipment & Medical Supplies',
                 'Eyewear Supplier',
                 'Home Infusion Therapy Pharmacy',
                 'Chiropractor',
                 'Pharmacy',
                 'Customized Equipment (DME)',
                 'Parenteral & Enteral Nutrition Supplies (DME)',
                 'Portable X-ray and/or Other Portable Diagnostic Imaging Supplier',
                 'Oral & Maxillofacial Surgery (D.M.D.)',
                 'Institutional Pharmacy',
                 'Exclusive Provider Organization',
                 'Mobile Radiology Clinic/Center',
                 'Public Health or Welfare Agency',
                 'Rehabilitation Chiropractor']

hospital_referrals = (
    hospital_referrals\
    .loc[~hospital_referrals['taxonomy_code_display_name_nppes_referral_to'].isin(exclude_these) &
         ~hospital_referrals['taxonomy_code_display_name_nppes_referral_from'].isin(exclude_these) &
        (hospital_referrals['taxonomy_code_classification_nppes_referral_from'] != 'Radiology') &
        (hospital_referrals['taxonomy_code_classification_nppes_referral_to'] != 'Radiology')]
)

In [11]:
# Set up affiliations through organization name, and then address matching.
hospital_referrals['affiliation_referral_to'] = ''
hospital_referrals['affiliation_referral_to']\
.loc[hospital_referrals['organization_name_nppes_referral_to']\
     .str.lower()\
     .str.contains('vand|maury regional')] = 'Vanderbilt'

hospital_referrals['affiliation_referral_to']\
.loc[hospital_referrals['organization_name_nppes_referral_to']\
     .str.lower()\
     .str.contains('thom|ascen')] = 'Saint Thomas'

hospital_referrals['affiliation_referral_to']\
.loc[hospital_referrals['organization_name_nppes_referral_to']\
     .str.lower()\
     .str.contains('hca|tristar|stonecrest|northcrest|skyline')] = 'HCA'

hospital_referrals['affiliation_referral_to']\
.loc[hospital_referrals['affiliation_referral_to'] == ''] = 'No Major Affiliation'

C:\Users\charr\AppData\Local\Temp/ipykernel_22384/4155539227.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hospital_referrals['affiliation_referral_to']\
C:\Users\charr\AppData\Local\Temp/ipykernel_22384/4155539227.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hospital_referrals['affiliation_referral_to']\
C:\Users\charr\AppData\Local\Temp/ipykernel_22384/4155539227.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hospi

In [12]:
hca_list = (
    hospital_referrals['address_first_line_nppes_referral_to']\
    .loc[hospital_referrals['affiliation_referral_to'] == 'HCA']\
    .map(lambda x: x.lower())\
    .unique()\
    .tolist()
)

vandy_list = (
    hospital_referrals['address_first_line_nppes_referral_to']\
    .loc[hospital_referrals['affiliation_referral_to'] == 'Vanderbilt']\
    .map(lambda x: x.lower())\
    .unique()\
    .tolist()
)

st_thomas_list = (
    hospital_referrals['address_first_line_nppes_referral_to']\
    .loc[hospital_referrals['affiliation_referral_to'] == 'Saint Thomas']\
    .map(lambda x: x.lower())\
    .unique()\
    .tolist()
)

In [13]:
hospital_referrals['affiliation_referral_to']\
.loc[hospital_referrals['address_first_line_nppes_referral_to']\
     .str.lower()\
     .isin(st_thomas_list)] = 'Saint Thomas'

hospital_referrals['affiliation_referral_to']\
.loc[hospital_referrals['address_first_line_nppes_referral_to']\
     .str.lower()\
     .isin(hca_list)] = 'HCA'

hospital_referrals['affiliation_referral_to']\
.loc[hospital_referrals['address_first_line_nppes_referral_to']\
     .str.lower()\
     .isin(vandy_list)] = 'Vanderbilt'

C:\Users\charr\AppData\Local\Temp/ipykernel_22384/2033757846.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hospital_referrals['affiliation_referral_to']\
C:\Users\charr\AppData\Local\Temp/ipykernel_22384/2033757846.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hospital_referrals['affiliation_referral_to']\
C:\Users\charr\AppData\Local\Temp/ipykernel_22384/2033757846.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hospi

In [14]:
hospital_referrals['affiliation_referral_from'] = ''

hospital_referrals['affiliation_referral_from']\
.loc[hospital_referrals['address_first_line_nppes_referral_from']\
     .str.lower()\
     .isin(st_thomas_list)] = 'Saint Thomas'

hospital_referrals['affiliation_referral_from']\
.loc[hospital_referrals['address_first_line_nppes_referral_from']\
     .str.lower()\
     .isin(hca_list)] = 'HCA'

hospital_referrals['affiliation_referral_from']\
.loc[hospital_referrals['address_first_line_nppes_referral_from']\
     .str.lower()\
     .isin(vandy_list)] = 'Vanderbilt'

hospital_referrals['affiliation_referral_from']\
.loc[hospital_referrals['affiliation_referral_from'] == ''] = 'No Major Affiliation'

C:\Users\charr\AppData\Local\Temp/ipykernel_22384/2134142636.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hospital_referrals['affiliation_referral_from']\
C:\Users\charr\AppData\Local\Temp/ipykernel_22384/2134142636.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hospital_referrals['affiliation_referral_from']\
C:\Users\charr\AppData\Local\Temp/ipykernel_22384/2134142636.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  h

In [15]:
hospital_referrals['affiliation_referral_from']\
.loc[hospital_referrals['address_first_line_nppes_referral_from']\
     .str.lower()\
     .str.contains('frist|35\\d+ new shackle island|3\\d{2} 23rd ave|stonecrest|39\\d{1} wallace|34\\d{2} dickerson|111 h|3901 central|24\\d{2} patterson|northcrest|310 25th|250 25th|redbud|recovery')] = 'HCA'

hospital_referrals['affiliation_referral_from']\
.loc[hospital_referrals['address_first_line_nppes_referral_from']\
     .str.lower()\
     .str.contains('vanderbilt|tvc|trotwood|43\\d{2} carothers|1211 medical center|1\\d{3} 21st ave|14\\d{2} w badd|1670 w main|854 w james|719 thompson|covey')] = 'Vanderbilt'

hospital_referrals['affiliation_referral_from']\
.loc[hospital_referrals['address_first_line_nppes_referral_from']\
     .str.lower()\
     .str.contains('18\\d{2} medical center|42\\d{2} harding|20th ave|20\\d{2} church st|1040 n james|2004 hayes street|landrum|doolittle|155 health')] = 'Saint Thomas'

hospital_referrals['affiliation_referral_from']\
.loc[hospital_referrals['address_first_line_nppes_referral_from']\
     .str.lower()\
     .str.contains('300 steam plant|555 hartsville')] = 'Sumner Regional'

hospital_referrals['affiliation_referral_from']\
.loc[hospital_referrals['address_first_line_nppes_referral_from']\
     .str.lower()\
     .str.contains('albion')] = 'Nashville General Hospital'

C:\Users\charr\AppData\Local\Temp/ipykernel_22384/89094668.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hospital_referrals['affiliation_referral_from']\
C:\Users\charr\AppData\Local\Temp/ipykernel_22384/89094668.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hospital_referrals['affiliation_referral_from']\
C:\Users\charr\AppData\Local\Temp/ipykernel_22384/89094668.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hospita

In [16]:
hospital_referrals['affiliation_referral_to']\
.loc[hospital_referrals['address_first_line_nppes_referral_to']\
     .str.lower()\
     .str.contains('frist|35\\d+ new shackle island|3\\d{2} 23rd ave|stonecrest|39\\d{1} wallace|34\\d{2} dickerson|111 h|3901 central|24\\d{2} patterson|northcrest|310 25th|250 25th|redbud|recovery')] = 'HCA'

hospital_referrals['affiliation_referral_to']\
.loc[hospital_referrals['address_first_line_nppes_referral_to']\
     .str.lower()\
     .str.contains('vanderbilt|tvc|trotwood|43\\d{2} carothers|1211 medical center|1\\d{3} 21st ave|14\\d{2} w badd|1670 w main|854 w james|719 thompson|covey')] = 'Vanderbilt'

hospital_referrals['affiliation_referral_to']\
.loc[hospital_referrals['address_first_line_nppes_referral_to']\
     .str.lower()\
     .str.contains('18\\d{2} medical center|42\\d{2} harding|20th ave|20\\d{2} church st|1040 n james|2004 hayes street|landrum|doolittle|155 health')] = 'Saint Thomas'

hospital_referrals['affiliation_referral_to']\
.loc[hospital_referrals['address_first_line_nppes_referral_to']\
     .str.lower()\
     .str.contains('300 steam plant|555 hartsville')] = 'Sumner Regional'

hospital_referrals['affiliation_referral_to']\
.loc[hospital_referrals['address_first_line_nppes_referral_to']\
     .str.lower()\
     .str.contains('albion')] = 'Nashville General Hospital'

C:\Users\charr\AppData\Local\Temp/ipykernel_22384/714364162.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hospital_referrals['affiliation_referral_to']\
C:\Users\charr\AppData\Local\Temp/ipykernel_22384/714364162.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hospital_referrals['affiliation_referral_to']\
C:\Users\charr\AppData\Local\Temp/ipykernel_22384/714364162.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hospital

In [17]:
# Concatenate strings in provider name and address and facility address columns. Rename some columns additionally.
upper_case_columns = ['affiliation_referral_from',
                      'affiliation_referral_to',
                      'taxonomy_code_grouping_nppes_referral_from',
                      'taxonomy_code_classification_nppes_referral_from',
                      'taxonomy_code_specialization_nppes_referral_from',
                      'taxonomy_code_display_name_nppes_referral_from',
                      'taxonomy_code_grouping_nppes_referral_to',
                      'taxonomy_code_classification_nppes_referral_to',
                      'taxonomy_code_specialization_nppes_referral_to',
                      'taxonomy_code_display_name_nppes_referral_to']

hospital_referrals[upper_case_columns] = hospital_referrals[upper_case_columns].apply(lambda x: x.str.upper())


hospital_referrals['provider_name'] = (
hospital_referrals[['name_prefix_nppes_referral_from',
                    'first_name_nppes_referral_from',
                    'middle_name_nppes_referral_from',
                    'last_name_nppes_referral_from',
                    'name_suffix_nppes_referral_from',
                    'credentials_nppes_referral_from']]\
    .astype(str)\
    .agg(' '.join, axis = 1)\
    .str.replace("nan ","")
)


hospital_referrals['provider_address'] = (
hospital_referrals[['address_first_line_nppes_referral_from',
                    'address_second_line_nppes_referral_from', 
                    'city_nppes_referral_from',
                    'state_nppes_referral_from', 
                    'zipcode_nppes_referral_from',]]\
    .astype(str)\
    .agg(' '.join, axis = 1)\
    .str.replace("nan ","")
)

hospital_referrals['facility_address'] = (
hospital_referrals[['address_first_line_nppes_referral_to',
                    'address_second_line_nppes_referral_to', 
                    'city_nppes_referral_to',
                    'state_nppes_referral_to', 
                    'zipcode_nppes_referral_to',]]\
    .astype(str)\
    .agg(' '.join, axis = 1)\
    .str.replace("nan ","")
)

hospital_referrals = (
    hospital_referrals.rename(columns = {'std_day_wait':'std_dev_day_wait',
                                         'patient_count':'number_of_patients',
                                         'transaction_count':'referrals',
                                         'npi_nppes_referral_from':'provider_npi',
                                         'affiliation_referral_from':'provider_affiliation',
                                         'taxonomy_code_nppes_referral_from':'provider_taxonomy_code',
                                         'taxonomy_code_grouping_nppes_referral_from':'provider_taxonomy_code_grouping',
                                         'taxonomy_code_classification_nppes_referral_from':'provider_taxonomy_code_classification',
                                         'taxonomy_code_specialization_nppes_referral_from':'provider_taxonomy_code_specialization',
                                         'taxonomy_code_display_name_nppes_referral_from':'provider_taxonomy_code_display_name',
                                         'npi_nppes_referral_to':'facility_npi',
                                         'affiliation_referral_to':'facility_affiliation',
                                         'organization_name_nppes_referral_to':'facility_entity_name',
                                         'taxonomy_code_nppes_referral_to':'facility_taxonomy_code',
                                         'taxonomy_code_grouping_nppes_referral_to':'facility_taxonomy_code_grouping',
                                         'taxonomy_code_classification_nppes_referral_to':'facility_taxonomy_code_classification',
                                         'taxonomy_code_specialization_nppes_referral_to':'facility_taxonomy_code_specialization',
                                         'taxonomy_code_display_name_nppes_referral_to':'facility_taxonomy_code_display_name'})
)

In [18]:
hospital_referrals = (
    hospital_referrals[['number_of_patients',
                        'referrals',
                        'average_day_wait',
                        'std_dev_day_wait',
                        'provider_npi',
                        'provider_affiliation',
                        'provider_name',
                        'provider_address',
                        'provider_taxonomy_code',
                        'provider_taxonomy_code_grouping',
                        'provider_taxonomy_code_classification',
                        'provider_taxonomy_code_specialization',
                        'provider_taxonomy_code_display_name',
                        'facility_npi',
                        'facility_affiliation',
                        'facility_entity_name',
                        'facility_address',
                        'facility_taxonomy_code',
                        'facility_taxonomy_code_grouping',
                        'facility_taxonomy_code_classification',
                        'facility_taxonomy_code_specialization',
                        'facility_taxonomy_code_display_name']]
)

In [19]:
# Write hospital referrals dataset to database and csv.
hospital_referrals.to_sql('hospital_referrals', db, index = False)  
hospital_referrals.to_csv('data/hospital_referrals.csv', index = False)